# How many half nights have I observed?

In [21]:
from bs4 import BeautifulSoup
from astropy.utils.data import download_file
import numpy as np

# Put the range years you'd like to include in the search
include_years = np.arange(2013, 2018)

# Put the first four letters of your last name 
# (and hope that they're unique at your institution)
observer_name = "Morr"

# If True, will save local copies of the webpages. This is
# useful if you'd like to run the script many times with 
# different names.
cache = True

In [22]:
text = download_file("http://www.apo.nmsu.edu/Planning/obs.calendar/obs.calendar.html", 
                     cache=cache, show_progress=False)
soup = BeautifulSoup(open(text).read(), 'lxml')

list_links = soup.body.findAll("a")

quarterly_links = []

for link in list_links:
    for year in include_years:
        url_root = "http://35m-schedule.apo.nmsu.edu/{0:04d}Q".format(year)
        if ('href' in link.attrs) and (url_root in link.get('href')): 
            quarterly_links.append(link.get('href'))

This part will take a few minutes to download all of the webpages: 

In [23]:
my_programs = dict()

for quarterly_link in quarterly_links:
    p = download_file(quarterly_link, cache=cache, show_progress=False)
    soup = BeautifulSoup(open(p).read(), 'html.parser')

    base_link = soup.head.base.get('href')

    uw_links = [link.get('href') for link in soup.body.findAll('a') 
                if (('href' in link.attrs) and link.get('href').startswith('programs/UW'))]

    uw_links = list(map(lambda x: "{0}{1}".format(base_link, x), uw_links))

    for uw_link in uw_links: 
        p = download_file(uw_link, cache=cache, show_progress=False)
        soup = BeautifulSoup(open(p).read(), 'html.parser')
        observed = observer_name in soup.body.text
        
        if observed: 
            program_id = soup.title.text
            quarter_id = soup.body.p.a.get('href').split('/')[-1][:6]
            n_nights = len([link.get('href') for link in soup.body.find_all('a') 
                            if 'days' in link.get('href')])
            my_programs['{0}_{1}'.format(quarter_id, program_id)] = n_nights

In [24]:
print("Total half nights:", sum(my_programs.values()))

Total half nights: 64


In [27]:
my_programs

{'2014Q4_UW03': 2,
 '2015Q1_UW02': 5,
 '2015Q2_UW04': 2,
 '2016Q1_UW01': 2,
 '2016Q3_UW01': 4,
 '2016Q3_UW04': 9,
 '2016Q4_UW13': 4,
 '2017Q1_UW01': 3,
 '2017Q2_UW01': 10,
 '2017Q3_UW01': 9,
 '2017Q3_UW03': 3,
 '2017Q4_UW01': 2,
 '2017Q4_UW06': 4,
 '2017Q4_UW09': 5}